# Evolving a PDF

In [1]:
import eko

Again, we need to recompute the operator, since we didn't save it.

In [2]:
from banana.data.theories import default_card as th_card
from ekomark.data.operators import default_card as op_card

# here we replace the grid with a very minimal one, to speed up the example
op_card["interpolation_xgrid"] = [1e-3, 1e-2, 1e-1, 5e-1, 1.]
# let's compute two operators: one reaching up to Q2=100 GeV^2 and a second reaching up to Q2=10000 GeV^2
op_card["Q2grid"] = [100., 10000.]
evolution_operator = eko.run_dglap(th_card, op_card)

Next, we need to load the PDF that we want to evolve. EKO uses the same interface as lhapdf to query for the actual values of PDFs. However, for the scope of this tutorial we want to avoid the complication of dealing with an external dependency. Therefore we will use the toy PDFs as they were established by the Les Houches benchmark setting.

In [3]:
from banana import toy
pdf = toy.mkPDF("",0)

Now, we have all ingredients at hand to evolve the PDF set with the operator:

In [4]:
from ekomark.apply import apply_pdf
evolved_pdfs = apply_pdf(evolution_operator, pdf)

The returned object `evolved_pdfs` is a dictionary which maps the requested final scales onto further dictionaries:

In [5]:
evolved_pdfs.keys()

dict_keys([100.0, 10000.0])

Each final scale contains a dictionary with a key `pdfs`, where all PDF values are hold, and a key `errors` , where the integration errors are hold.
Finally, inside each of those there is a mapping of Monte Carlo particle identifiers onto a the PDF value at the requested interpolation points.

E.g. to access the gluon PDF at `Q^2 = 100 GeV^2` you can run:

In [6]:
evolved_pdfs[100.0]["pdfs"][21]

array([ 2.17034797e+04,  3.40247532e+02,  6.64325189e+01, -2.09652559e+01,
        0.00000000e+00])

Note that we return the actual PDF and not the momentum fraction times the PDF.